# XGBoost for Whale Classification

In this assignment, you have been provided a dataset that consists of echo-location clicks of two types of whales, namely, Gervais and Cuviers. Your task is to classify the different types whales using Gradient Boosting with the help of the XGBoost library. You are expected to fill in functions that would complete this task. We use XGBoost here instead of GradientBoostedTrees in Spark because XGBoost running on a single machine is much faster than Spark running on 10 machines.

The data files were preprocessed on PySpark (10 nodes) cluster. The code for the same can be found in Data_Processing_Whales.ipynb. The preprocessed data is a numpy array with `4175` rows (for the 10mb file) with following columns (zero-indexed):
* Col 0-9: projections on first 10 eigen vectors
* Col 10: rmse
* Col 11: peak2peak
* Col 12: label (`0 if row.species==u'Gervais' else 1`)

You can also refer to XGBoost_Whales.ipynb under for more details on the XGBoost Analysis before you attempt this assignment.

Both Data_Processing_Whales.ipynb and XGBoost_Whales.ipynb can be found under Lecture Notebooks Spring 2023/Section3-Classification/XGBoost directory in Vocareum.

## XGBoost - Theory

A brief overview of gradient boosting in XGBoost can be found here:

* https://xgboost.readthedocs.io/en/latest/tutorials/model.html
* https://xgboost.readthedocs.io/en/latest/python/python_intro.html

Use the XGBoost API for training and predicting scores: 

* http://xgboost.readthedocs.io/en/latest/python/python_api.html

#### Main API

* `xgboost.train` is the learning API that trains the Gradient Boosting Model,
   * The main parameters are:
      * **plst** – XGBoost parameter list
      * **dtrain** – Data to be trained
      * **num_round** – Number of iterations of boosting. (default: 100)
      * **evallist** – List of items to be evaluated during training
      * **verbose_eval** - This can be used to control how much information the train function prints. You might want to set to **False** to avoid printing logs.
* `bst.predict` is the API that makes score predictions
   * The main parameters are:
      * **dtest** – Test Data
      * **dtrain** – Data to be trained
      * **ntree_limit** – Limit number of trees in the prediction (Use: ntree_limit=bst.best_ntree_limit)
      * **output_margin** - Whether to output the raw untransformed margin value (Use: output_margin=True)

## Notebook Setup

### Importing Required Libraries

In [1]:
%matplotlib inline
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import random

In [2]:
xgb.__version__

'0.72'

### Loading Data

In [3]:
with open('./resource/asnlib/publicdata/X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open('./resource/asnlib/publicdata/X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open('./resource/asnlib/publicdata/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('./resource/asnlib/publicdata/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

### Setting Parameters for XG Boost
* Maximum Depth of the Tree = 3 _(maximum depth of each decision trees)_
* Step size shrinkage used in update to prevents overfitting = 0.3 _(how to weigh trees in subsequent iterations)_
* Evaluation Criterion= Maximize Loglikelihood according to the logistic regression _(logitboost)_
* Maximum Number of Iterations = 1000 _(total number trees for boosting)_
* Early Stop if score on Validation does not improve for 5 iterations

[Full description of options](https://xgboost.readthedocs.io/en/latest/parameter.html)

In [4]:
#You can change this cell if you wish to, but you aren't expected to
def xgboost_plst():
    param = {}
    param['max_depth']= 3   # depth of tree
    param['eta'] = 0.3      # shrinkage parameter
    param['silent'] = 1     # not silent
    param['objective'] = 'binary:logistic'
    param['nthread'] = 7 # Number of threads used
    param['eval_metric'] = 'logloss'

    plst = param.items()
    return plst

## Exercises

### Computing the score ranges

The function `calc_stats` takes the xgboost margin scores as input and returns two numpy arrays `min_scr`, `max_scr` which are calculated as follows:

1. `min_scr`: mean - (3 $\times$ std)
2. `max_scr`: mean + (3 $\times$ std)

Here the input margin scores, represents the processed XGBoost margin scores obtained from the `bootstrap_pred` function. Each row represents the various scores for a specific example in an iteration and your `calc_stats` function is supposed to compute the `min_scr` and `max_scr` as defined for each example. So in the example below, we take a scenario where we have 3 examples which have 4 values each (From 4 bootstrap iterations).

#### Tasks:

Finish the function `calc_stats`.

Input:

- `margin_scores`: a 2d numpy array that contains xgboost margin scores. 

Output: 

- 2 numpy arrays of `min_scr` and `max_scr` as defined above. 


**Note**: Ensure you round the values in the numpy arrays to two decimal places

---

**<font color="magenta" size=2>Example Input</font>**
``` python
[[-0.22 -0.19 -0.17 -0.13][-0.1 -0.05 0.02 0.10][0.03 0.11 0.12 0.15]]
```


**<font color="blue" size=2>Example Output</font>**
``` python
(array([-0.28 -0.23 -0.03]),
 array([-0.08  0.22  0.24]))
```

In [9]:
def calc_stats(margin_scores):
    ###
    margin_scores = np.array(margin_scores)
    mean = np.mean(margin_scores, axis=1)
    standard_deviation = np.std(margin_scores, axis=1)
    
    min_scr = mean-3*standard_deviation
    max_scr = mean+3*standard_deviation
    
    min_scr = np.round(min_scr, 2)
    max_scr = np.round(max_scr, 2)
    return min_scr, max_scr
    ###


In [10]:
margin_score = np.array([[-0.22, -0.19, -0.17, -0.13], [-0.1, -0.05, 0.02, 0.10], [0.03, 0.11, 0.12, 0.15]])
min_score, max_score = calc_stats(margin_score)
assert type(min_score) == np.ndarray, 'Incorrect Return type'
assert type(max_score) == np.ndarray, 'Incorrect Return type'

In [11]:
assert (min_score == np.array([-0.28, -0.23, -0.03])).all(), "Incorrect return value"

In [12]:
assert (max_score == np.array([-0.08,  0.22,  0.24])).all(), "Incorrect return value"

In [13]:
# Hidden Tests Here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [14]:
# Hidden Tests Here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


### Calculating predictions

Based on the ranges for each of the examples, i.e, (`min_scr`, `max_scr`), we can predict whether it's a Gervais or a Cuvier. Since all our scores will be between -1 and +1, we use 0 as the margin line. All examples which are on the left side of the margin, can be classified as Cuvier's and all which are on the right side can be classified as Gervais. However, since we take margin scores from a set of bootstraps for each example, we use the minimum and maximum score arrays to predict to determine the classification.

#### Tasks:

Finish the function `predict`, which takes the minimum score array and maximum score array and returns predictions as follows:

1. If respective minimum score and maximum score values are less than 0, predict -1 (**Cuvier's**)
2. If respective minimum score value is less than 0 and maximum score value is greater than 0, predict 0 (**Unsure**)
3. If respective minimum score and maximum score values are greater than 0, predict 1 (**Gervais**)


Input:

- `min_scr`: the numpy array from `calc_stats`

- `max_scr`: the numpy array from `calc_stats`

Output: 

- a numpy array of predictions

---

**<font color="magenta" size=2>Example Input</font>**
``` python
min_scr (numpy array) = [-0.78 -0.68 -0.6 -0.53 -0.47 -0.42 -0.32 -0.21 -0.07 0.22]

max_scr (numpy array) = [-0.49 -0.39 -0.33 -0.25 -0.2 -0.11 -0.04 0.1 0.3 0.51]
```

**<font color="blue" size=2>Example Output</font>**
``` python
[-1 -1 -1 -1 -1 -1 -1  0  0  1]
```

In [17]:
def predict(min_scr, max_scr):
    ###
    predictions = np.zeros(min_scr.shape, dtype=int)
    predictions[(min_scr < 0) & (max_scr < 0)] = -1
    predictions[(min_scr > 0) & (max_scr > 0)] = 1
    
    return predictions
    ###


In [18]:
max_s = np.array([-0.49, -0.39, -0.33, -0.25, -0.2, -0.11, -0.04, 0.1, 0.3, 0.51])
min_s = np.array([-0.78, -0.68, -0.6, -0.53, -0.47, -0.42, -0.32, -0.21, -0.07, 0.22])
pred = predict(min_s, max_s)
true_pred = np.array([-1, -1, -1, -1, -1, -1, -1, 0, 0, 1])

In [19]:
assert type(pred) == np.ndarray, 'Incorrect return type'

In [20]:
assert (pred == true_pred).all(), 'Incorrect return value'

### Calculating scores

You can follow these procedures to train a number of XGBoost models using bootstrap and test their performances.

Repeat the given procedure for `n_bootstrap` number of iterations:

For `n_bootstrap` iterations:
* Sample `boostrap_size` indices from the training set **with replacmennt**.
* Create train and test data matrices `(dtrain, dtest)` using `xgb.DMatrix(X_sample, label=y_sample)`.
* Initialise the `evallist` parameter `[(dtrain, 'train'), (dtest, 'eval')]`.
* Train the model using the XGBoost train API and make score predictions using bst.predict object returned by XGB train API. **Ensure you set `output_margin=True` to get raw untransformed output scores and `ntree_limit=bst.best_ntree_limit`**.
* Normalize them by dividing them with the normalizing factor as `max(scores) - min(scores)` and round these values to a precision of two decimal places.

Then: 
* For each individual example, remove scores below the minRth score and greater than the maxRth score(sort for each example if necessary).
* Call the `calc_stats` function to compute `min_scr` and `max_scr` with the filtered margin scores as parameter.
* Return the `min_scr` and `max_scr` computed by the `calc_stat` function using the margin scores.

#### Task:

Finish the function `bootstrap_pred`.

Input:

- `Training set`: the training set.
- `Test set`: the test test.
- `n_bootstrap`: number of bootstrap samples that run XGBoost and trains one part of the sample set.
- `minR, maxR`: two numbers such that $0 < minR < maxR < 1$ that define the fractions of the `n_bootstrap` scores that define the range.
- `bootstrap_size`: number of bootstrap samples on which you will run XGBoost.
- `num_round`: number of iterations for running xgboost.
- `plst`: parameter List.

Output:

- The output should be a confidence interval for each example in the test set (`min_scr` and `max_scr`). 

**Note**: 

- Please see the [Main API](#Main-API) section above to see how to use XGBoost.
- Remember to set `num_round` and `plst` as specified by the parameters in `xgb.train`.
- You can experiment by changing `n_bootstraps`, but it takes about 200 iterations to get consistent values.

In [44]:
def bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap, minR, maxR, bootstrap_size, \
                   num_round=100, plst=xgboost_plst()):
    ###
    try:
        scores_array = np.zeros((X_test.shape[0], n_bootstrap))
    
        for i in range(n_bootstrap):
            #X_sample, y_sample = resample(X_train, y_train, n_samples=bootstrap_size)

            indices= np.random.choice(X_train.shape[0], size=bootstrap_size, replace=True)
            X_sample = X_train[indices]
            y_sample = y_train[indices]

            dtrain = xgb.DMatrix(X_sample, label=y_sample)
            dtest = xgb.DMatrix(X_test)

            eval_list = [(dtrain, 'train')]
            bst = xgb.train(plst, dtrain, num_round, eval_list)
            
            scores = bst.predict(dtest, output_margin=True, ntree_limit=bst.best_ntree_limit)

            norm_factor = scores.max() - scores.min()
            normalized_scores = (scores - scores.min()) / norm_factor if norm_factor else scores
            
            scores_array[:,i] = scores/norm_factor
        
        sorted_scores = np.sort(scores_array, axis=1)
        low_idx = int(np.floor(minR * n_bootstrap))
        high_idx = int(np.floor(maxR * n_bootstrap))
        min_scr = np.min(sorted_scores[:, low_idx:high_idx], axis=1)
        max_scr = np.max(sorted_scores[:, low_idx:high_idx], axis=1)
        
        return min_scr, max_scr
    
    except Exception as e:
        print(f"Error in bootstrap_pred: {e}")
        return None, None
    
    
    ###


In [45]:
def process(X_train, X_test, y_train, y_test, n_bootstrap=100):
    min_scr, max_scr = bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap=n_bootstrap, \
                                            minR=0.1, maxR=0.9, bootstrap_size=len(X_train))
    pred = predict(min_scr, max_scr)
    return min_scr, max_scr, pred

#### Tests

How we test the function:
1. We have computed the average mid-point of the range of values and verify that this midpoint is present in the range computed by your function
2. We check that the length of the `interval(max_scr-min_scr)` is not more than twice the average length of the interval

In [46]:
sample_indices = np.load('./resource/asnlib/publicdata/vis_indices.npy')
X_test_samp = X_test[sample_indices]
y_test_samp = np.array(y_test[sample_indices], dtype=int)
midpt = np.load('./resource/asnlib/publicdata/vis_midpt.npy')
avg_length = np.load('./resource/asnlib/publicdata/vis_avg_length.npy')
min_scr, max_scr, pred = process(X_train, X_test_samp, y_train, y_test_samp)
length = max_scr - min_scr

[0]	train-logloss:0.556831
[1]	train-logloss:0.476768
[2]	train-logloss:0.426502
[3]	train-logloss:0.392629
[4]	train-logloss:0.368913
[5]	train-logloss:0.349772
[6]	train-logloss:0.333541
[7]	train-logloss:0.319452
[8]	train-logloss:0.309243
[9]	train-logloss:0.299066
[10]	train-logloss:0.291291
[11]	train-logloss:0.283998
[12]	train-logloss:0.276145
[13]	train-logloss:0.271258
[14]	train-logloss:0.265263
[15]	train-logloss:0.258503
[16]	train-logloss:0.253449
[17]	train-logloss:0.249808
[18]	train-logloss:0.244528
[19]	train-logloss:0.239902
[20]	train-logloss:0.237995
[21]	train-logloss:0.232034
[22]	train-logloss:0.228647
[23]	train-logloss:0.223251
[24]	train-logloss:0.220678
[25]	train-logloss:0.217918
[26]	train-logloss:0.215929
[27]	train-logloss:0.212323
[28]	train-logloss:0.210438
[29]	train-logloss:0.206436
[30]	train-logloss:0.204861
[31]	train-logloss:0.202128
[32]	train-logloss:0.200759
[33]	train-logloss:0.198613
[34]	train-logloss:0.195422
[35]	train-logloss:0.193691
[3

[95]	train-logloss:0.099207
[96]	train-logloss:0.097992
[97]	train-logloss:0.097694
[98]	train-logloss:0.097182
[99]	train-logloss:0.096219
[0]	train-logloss:0.554955
[1]	train-logloss:0.476886
[2]	train-logloss:0.428244
[3]	train-logloss:0.396838
[4]	train-logloss:0.373598
[5]	train-logloss:0.357364
[6]	train-logloss:0.343752
[7]	train-logloss:0.331406
[8]	train-logloss:0.321963
[9]	train-logloss:0.311749
[10]	train-logloss:0.303882
[11]	train-logloss:0.2966
[12]	train-logloss:0.291607
[13]	train-logloss:0.287708
[14]	train-logloss:0.280747
[15]	train-logloss:0.276993
[16]	train-logloss:0.27055
[17]	train-logloss:0.265201
[18]	train-logloss:0.262242
[19]	train-logloss:0.257487
[20]	train-logloss:0.255114
[21]	train-logloss:0.251471
[22]	train-logloss:0.247048
[23]	train-logloss:0.244996
[24]	train-logloss:0.240386
[25]	train-logloss:0.236237
[26]	train-logloss:0.231438
[27]	train-logloss:0.22803
[28]	train-logloss:0.223425
[29]	train-logloss:0.219299
[30]	train-logloss:0.215192
[31]	t

[90]	train-logloss:0.109865
[91]	train-logloss:0.10926
[92]	train-logloss:0.108806
[93]	train-logloss:0.108331
[94]	train-logloss:0.107778
[95]	train-logloss:0.106259
[96]	train-logloss:0.105424
[97]	train-logloss:0.103715
[98]	train-logloss:0.102942
[99]	train-logloss:0.101805
[0]	train-logloss:0.553716
[1]	train-logloss:0.473875
[2]	train-logloss:0.424304
[3]	train-logloss:0.390859
[4]	train-logloss:0.367921
[5]	train-logloss:0.351279
[6]	train-logloss:0.335597
[7]	train-logloss:0.321933
[8]	train-logloss:0.314044
[9]	train-logloss:0.304382
[10]	train-logloss:0.297997
[11]	train-logloss:0.289063
[12]	train-logloss:0.282936
[13]	train-logloss:0.277826
[14]	train-logloss:0.27224
[15]	train-logloss:0.265855
[16]	train-logloss:0.261689
[17]	train-logloss:0.259121
[18]	train-logloss:0.255819
[19]	train-logloss:0.249765
[20]	train-logloss:0.244087
[21]	train-logloss:0.239806
[22]	train-logloss:0.234388
[23]	train-logloss:0.230187
[24]	train-logloss:0.227773
[25]	train-logloss:0.226458
[26]

[85]	train-logloss:0.106868
[86]	train-logloss:0.10517
[87]	train-logloss:0.103781
[88]	train-logloss:0.102778
[89]	train-logloss:0.101951
[90]	train-logloss:0.100882
[91]	train-logloss:0.100045
[92]	train-logloss:0.099515
[93]	train-logloss:0.098562
[94]	train-logloss:0.097537
[95]	train-logloss:0.096024
[96]	train-logloss:0.094466
[97]	train-logloss:0.093976
[98]	train-logloss:0.09271
[99]	train-logloss:0.092336
[0]	train-logloss:0.55534
[1]	train-logloss:0.474885
[2]	train-logloss:0.425353
[3]	train-logloss:0.391986
[4]	train-logloss:0.368085
[5]	train-logloss:0.350669
[6]	train-logloss:0.33619
[7]	train-logloss:0.324775
[8]	train-logloss:0.313811
[9]	train-logloss:0.30469
[10]	train-logloss:0.297516
[11]	train-logloss:0.290189
[12]	train-logloss:0.283747
[13]	train-logloss:0.280204
[14]	train-logloss:0.273364
[15]	train-logloss:0.266521
[16]	train-logloss:0.260785
[17]	train-logloss:0.25579
[18]	train-logloss:0.252774
[19]	train-logloss:0.246134
[20]	train-logloss:0.241458
[21]	tra

[80]	train-logloss:0.11822
[81]	train-logloss:0.116738
[82]	train-logloss:0.115002
[83]	train-logloss:0.113902
[84]	train-logloss:0.112815
[85]	train-logloss:0.110944
[86]	train-logloss:0.109863
[87]	train-logloss:0.109065
[88]	train-logloss:0.107821
[89]	train-logloss:0.107135
[90]	train-logloss:0.105642
[91]	train-logloss:0.104451
[92]	train-logloss:0.103268
[93]	train-logloss:0.102155
[94]	train-logloss:0.10105
[95]	train-logloss:0.099648
[96]	train-logloss:0.098471
[97]	train-logloss:0.097832
[98]	train-logloss:0.097412
[99]	train-logloss:0.096208
[0]	train-logloss:0.549746
[1]	train-logloss:0.469703
[2]	train-logloss:0.416482
[3]	train-logloss:0.382524
[4]	train-logloss:0.355375
[5]	train-logloss:0.337011
[6]	train-logloss:0.320839
[7]	train-logloss:0.306408
[8]	train-logloss:0.296157
[9]	train-logloss:0.2859
[10]	train-logloss:0.278003
[11]	train-logloss:0.270557
[12]	train-logloss:0.264858
[13]	train-logloss:0.257502
[14]	train-logloss:0.251792
[15]	train-logloss:0.246696
[16]	t

...

[57]	train-logloss:0.155987
[58]	train-logloss:0.155085
[59]	train-logloss:0.152589
[60]	train-logloss:0.150122
[61]	train-logloss:0.148255
[62]	train-logloss:0.146235
[63]	train-logloss:0.143896
[64]	train-logloss:0.141303
[65]	train-logloss:0.140671
[66]	train-logloss:0.139245
[67]	train-logloss:0.137803
[68]	train-logloss:0.135916
[69]	train-logloss:0.134081
[70]	train-logloss:0.131929
[71]	train-logloss:0.130035
[72]	train-logloss:0.129204
[73]	train-logloss:0.128621
[74]	train-logloss:0.126455
[75]	train-logloss:0.124758
[76]	train-logloss:0.123968
[77]	train-logloss:0.122736
[78]	train-logloss:0.120857
[79]	train-logloss:0.119854
[80]	train-logloss:0.119299
[81]	train-logloss:0.118245
[82]	train-logloss:0.117102
[83]	train-logloss:0.116635
[84]	train-logloss:0.116158
[85]	train-logloss:0.115733
[86]	train-logloss:0.115121
[87]	train-logloss:0.11422
[88]	train-logloss:0.112532
[89]	train-logloss:0.111945
[90]	train-logloss:0.111375
[91]	train-logloss:0.110703
[92]	train-logloss:0.

[52]	train-logloss:0.167631
[53]	train-logloss:0.165853
[54]	train-logloss:0.163972
[55]	train-logloss:0.163152
[56]	train-logloss:0.162327
[57]	train-logloss:0.161354
[58]	train-logloss:0.160424
[59]	train-logloss:0.157208
[60]	train-logloss:0.154645
[61]	train-logloss:0.153785
[62]	train-logloss:0.153054
[63]	train-logloss:0.152507
[64]	train-logloss:0.152095
[65]	train-logloss:0.151532
[66]	train-logloss:0.150407
[67]	train-logloss:0.149168
[68]	train-logloss:0.148528
[69]	train-logloss:0.148047
[70]	train-logloss:0.146672
[71]	train-logloss:0.145705
[72]	train-logloss:0.143616
[73]	train-logloss:0.143316
[74]	train-logloss:0.141621
[75]	train-logloss:0.139897
[76]	train-logloss:0.137856
[77]	train-logloss:0.137268
[78]	train-logloss:0.135434
[79]	train-logloss:0.133057
[80]	train-logloss:0.131872
[81]	train-logloss:0.131418
[82]	train-logloss:0.130477
[83]	train-logloss:0.12998
[84]	train-logloss:0.129427
[85]	train-logloss:0.128314
[86]	train-logloss:0.1264
[87]	train-logloss:0.12

[47]	train-logloss:0.165101
[48]	train-logloss:0.162779
[49]	train-logloss:0.16043
[50]	train-logloss:0.159681
[51]	train-logloss:0.158224
[52]	train-logloss:0.156209
[53]	train-logloss:0.155465
[54]	train-logloss:0.153351
[55]	train-logloss:0.150792
[56]	train-logloss:0.148119
[57]	train-logloss:0.145954
[58]	train-logloss:0.14446
[59]	train-logloss:0.144064
[60]	train-logloss:0.143182
[61]	train-logloss:0.141915
[62]	train-logloss:0.140645
[63]	train-logloss:0.138905
[64]	train-logloss:0.136936
[65]	train-logloss:0.13611
[66]	train-logloss:0.134219
[67]	train-logloss:0.132706
[68]	train-logloss:0.131011
[69]	train-logloss:0.129449
[70]	train-logloss:0.128756
[71]	train-logloss:0.12774
[72]	train-logloss:0.126897
[73]	train-logloss:0.125208
[74]	train-logloss:0.123273
[75]	train-logloss:0.121946
[76]	train-logloss:0.12152
[77]	train-logloss:0.12023
[78]	train-logloss:0.119335
[79]	train-logloss:0.11875
[80]	train-logloss:0.117734
[81]	train-logloss:0.116566
[82]	train-logloss:0.11506


[42]	train-logloss:0.185728
[43]	train-logloss:0.183444
[44]	train-logloss:0.181634
[45]	train-logloss:0.179124
[46]	train-logloss:0.17708
[47]	train-logloss:0.174447
[48]	train-logloss:0.171806
[49]	train-logloss:0.169682
[50]	train-logloss:0.167657
[51]	train-logloss:0.167079
[52]	train-logloss:0.16611
[53]	train-logloss:0.16366
[54]	train-logloss:0.16257
[55]	train-logloss:0.160954
[56]	train-logloss:0.159627
[57]	train-logloss:0.158812
[58]	train-logloss:0.157359
[59]	train-logloss:0.155669
[60]	train-logloss:0.153798
[61]	train-logloss:0.152969
[62]	train-logloss:0.151103
[63]	train-logloss:0.14889
[64]	train-logloss:0.148112
[65]	train-logloss:0.147387
[66]	train-logloss:0.145533
[67]	train-logloss:0.143957
[68]	train-logloss:0.142407
[69]	train-logloss:0.140184
[70]	train-logloss:0.138178
[71]	train-logloss:0.136168
[72]	train-logloss:0.134679
[73]	train-logloss:0.133101
[74]	train-logloss:0.13214
[75]	train-logloss:0.131283
[76]	train-logloss:0.129375
[77]	train-logloss:0.12800

[37]	train-logloss:0.196945
[38]	train-logloss:0.195978
[39]	train-logloss:0.194664
[40]	train-logloss:0.191498
[41]	train-logloss:0.190328
[42]	train-logloss:0.188844
[43]	train-logloss:0.187634
[44]	train-logloss:0.186102
[45]	train-logloss:0.183883
[46]	train-logloss:0.182868
[47]	train-logloss:0.181426
[48]	train-logloss:0.180428
[49]	train-logloss:0.178183
[50]	train-logloss:0.175189
[51]	train-logloss:0.173758
[52]	train-logloss:0.173084
[53]	train-logloss:0.171795
[54]	train-logloss:0.17035
[55]	train-logloss:0.169962
[56]	train-logloss:0.168722
[57]	train-logloss:0.165327
[58]	train-logloss:0.162866
[59]	train-logloss:0.160859
[60]	train-logloss:0.157897
[61]	train-logloss:0.156226
[62]	train-logloss:0.154249
[63]	train-logloss:0.151956
[64]	train-logloss:0.151024
[65]	train-logloss:0.149724
[66]	train-logloss:0.148405
[67]	train-logloss:0.146434
[68]	train-logloss:0.145694
[69]	train-logloss:0.144182
[70]	train-logloss:0.142211
[71]	train-logloss:0.140614
[72]	train-logloss:0.

In [47]:
assert sum(min_scr <= midpt) >= (0.7 * len(sample_indices)), "Incorrect range (mean - 3*std) to (mean + 3*std)"

In [48]:
assert sum(max_scr >= midpt) >= (0.7 * len(sample_indices)), "Incorrect range (mean - 3*std) to (mean + 3*std)"

In [49]:
assert sum(length < 2*avg_length) >= (0.7 * len(sample_indices)), "Incorrect length of range (mean - 3*std) to (mean + 3*std)"

In [50]:
# Hidden Tests Here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [51]:
# Hidden Tests Here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [52]:
# Hidden Tests Here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [53]:
# Hidden Tests Here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [54]:
# Hidden Tests here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [55]:
# Hidden Tests here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [56]:
# Hidden Tests here
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [57]:
# Hidden Tests here
###
### AUTOGRADER TEST - DO NOT REMOVE
###
